In [210]:
#standard imports. Note: You must pip install nasdaqdatalink 1st

import os
import pandas as pd
import nasdaqdatalink
import requests
from pathlib import Path
import shutil

In [211]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 

nasdaqdatalink.read_key(filename=".env")

In [212]:
#A function to retrieve a dataframe of counties, zips, etc
def get_regions(regions):
    region_df=nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type=regions)  
    return region_df

In [213]:
# Using get_regions to retrieve a list of counties
region_df = get_regions('county')
region_df

,region_id,region_type,region
None,,,
0,999,county,Durham County; NC; Durham-Chapel Hill
1,998,county,Duplin County; NC
2,997,county,Dubois County; IN; Jasper
3,995,county,Donley County; TX
4,993,county,Dimmit County; TX
...,...,...,...
2886,1003,county,Elmore County; AL; Montgomery
2887,1002,county,Elbert County; GA
2888,1001,county,Elbert County; CO; Denver-Aurora-Lakewood


In [271]:
# Massaging zillow county names

region_df['region']=region_df['region'].str.split(';', n=-1, expand=True)[0]

In [272]:
print(region_df)

      region_id region_type         region
None                                      
0           999      county  Durham County
1           998      county  Duplin County
2           997      county  Dubois County
3           995      county  Donley County
4           993      county  Dimmit County
...         ...         ...            ...
2886       1003      county  Elmore County
2887       1002      county  Elbert County
2888       1001      county  Elbert County
2889       1000      county  Echols County
2890        100      county    Bibb County

[2891 rows x 3 columns]


In [216]:
#list(region_df['region_id'])

In [9]:
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink

# '''data = nasdaqdatalink.get_table('ZILLOW/DATA', paginate=True, indicator_id='ZSFH', region_id=list(region_df['region_id']))'''

data = nasdaqdatalink.export_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=list(region_df['region_id']),filename='db.zip')

DataLinkError: (Status 413) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.

In [180]:
# Unzipping database from API call

shutil.unpack_archive('db.zip')

In [278]:
# Reading in Database

zillow_data=pd.read_csv(
    Path('ZILLOW_DATA_d5d2ff90eb7172dbde848ea36de12dfe.csv')
)

In [279]:
# Displaying database

display(zillow_data.head(10))
display(zillow_data.tail(10))

,indicator_id,region_id,date,value
0,ZSFH,100,2007-11-30,123760.0
1,ZSFH,100,2007-12-31,123754.0
2,ZSFH,100,2008-01-31,123605.0
3,ZSFH,100,2008-02-29,123393.0
4,ZSFH,100,2008-03-31,123095.0
5,ZSFH,100,2008-04-30,123018.0
6,ZSFH,100,2008-05-31,122855.0
7,ZSFH,100,2008-06-30,122883.0
8,ZSFH,100,2008-07-31,122853.0
9,ZSFH,100,2008-08-31,123015.0


,indicator_id,region_id,date,value
669306,ZSFH,999,2021-09-30,346041.0
669307,ZSFH,999,2021-10-31,355160.0
669308,ZSFH,999,2021-11-30,361432.0
669309,ZSFH,999,2021-12-31,368325.0
669310,ZSFH,999,2022-01-31,380061.0
669311,ZSFH,999,2022-02-28,390111.0
669312,ZSFH,999,2022-03-31,401621.0
669313,ZSFH,999,2022-04-30,411421.0
669314,ZSFH,999,2022-05-31,422028.0
669315,ZSFH,999,2022-06-30,430509.0


In [280]:
# Dataframe information

display(zillow_data.head())
display(zillow_data.info())
display(region_df.head())
display(region_df.info())

,indicator_id,region_id,date,value
0,ZSFH,100,2007-11-30,123760.0
1,ZSFH,100,2007-12-31,123754.0
2,ZSFH,100,2008-01-31,123605.0
3,ZSFH,100,2008-02-29,123393.0
4,ZSFH,100,2008-03-31,123095.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 669316 entries, 0 to 669315
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   indicator_id  669316 non-null  object 
 1   region_id     669316 non-null  int64  
 2   date          669316 non-null  object 
 3   value         669316 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 20.4+ MB


None

,region_id,region_type,region
None,,,
0,999,county,Durham County
1,998,county,Duplin County
2,997,county,Dubois County
3,995,county,Donley County
4,993,county,Dimmit County


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   region_id    2891 non-null   int32 
 1   region_type  2891 non-null   object
 2   region       2891 non-null   object
dtypes: int32(1), object(2)
memory usage: 56.6+ KB


None

In [281]:
# CORRECT JOINING OF COUNTY AND ZILLOW DATAFRAMES

region_df['region_id']=region_df['region_id'].astype(int)
data_county_df = pd.merge(region_df, zillow_data, on='region_id')

data_county_df

,region_id,region_type,region,indicator_id,date,value
0,999,county,Durham County,ZSFH,1997-02-28,139430.0
1,999,county,Durham County,ZSFH,1997-03-31,139459.0
2,999,county,Durham County,ZSFH,1997-04-30,139659.0
3,999,county,Durham County,ZSFH,1997-05-31,139887.0
4,999,county,Durham County,ZSFH,1997-06-30,140303.0
...,...,...,...,...,...,...
669311,100,county,Bibb County,ZSFH,2022-02-28,161462.0
669312,100,county,Bibb County,ZSFH,2022-03-31,162369.0
669313,100,county,Bibb County,ZSFH,2022-04-30,163859.0
669314,100,county,Bibb County,ZSFH,2022-05-31,164684.0


In [282]:
# Read in county data with coordinates

county_coordinates_df=pd.read_csv(
    Path('counties_w_coordinates.csv')
)

county_coordinates_df

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,+32.536382°,–86.644490°
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",+30.659218°,–87.746067°
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,+31.870670°,–85.405456°
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3139,3139.0,WY,56037.0,Sweetwater,Green River,"43,806","27,004.90","10,426.65",166.887,64.436,"27,171.78","10,491.09",+41.660339°,–108.875676°
3140,3140.0,WY,56039.0,Teton,Jackson,"21,294","10,347.98","3,995.38",572.266,220.953,"10,920.25","4,216.33",+44.049321°,–110.588102°
3141,3141.0,WY,56041.0,Uinta,Evanston,"21,118","5,390.45","2,081.26",16.342,6.31,"5,406.79","2,087.57",+41.284726°,–110.558947°
3142,3142.0,WY,56043.0,Washakie,Worland,"8,533","5,797.82","2,238.55",10.762,4.155,"5,808.58","2,242.70",+43.878831°,–107.669052°


In [283]:
# Dataframe info

display(county_coordinates_df.info())
display(data_county_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3144 entries, 0 to 3143
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sort [1]            3143 non-null   float64
 1   State               3143 non-null   object 
 2   FIPS                3143 non-null   float64
 3   County [2]          3143 non-null   object 
 4   County Seat(s) [3]  3143 non-null   object 
 5   Population          3144 non-null   object 
 6   Land Area           3144 non-null   object 
 7   Land Area.1         3144 non-null   object 
 8   Water Area          3144 non-null   object 
 9   Water Area.1        3144 non-null   object 
 10  Total Area          3144 non-null   object 
 11  Total Area.1        3144 non-null   object 
 12  Latitude            3143 non-null   object 
 13  Longitude           3143 non-null   object 
dtypes: float64(2), object(12)
memory usage: 344.0+ KB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 669316 entries, 0 to 669315
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   region_id     669316 non-null  int32  
 1   region_type   669316 non-null  object 
 2   region        669316 non-null  object 
 3   indicator_id  669316 non-null  object 
 4   date          669316 non-null  object 
 5   value         669316 non-null  float64
dtypes: float64(1), int32(1), object(4)
memory usage: 33.2+ MB


None

In [286]:
# Column renaming

data_county_df = data_county_df.rename(columns={"region" : "county"})
county_coordinates_df = county_coordinates_df.rename(columns={"County\xa0[2]" : "county"})

display(county_coordinates_df.head())
display(data_county_df.head())

,Sort [1],State,FIPS,county,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,+32.536382°,–86.644490°
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",+30.659218°,–87.746067°
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,+31.870670°,–85.405456°
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°


,region_id,region_type,county,indicator_id,date,value
0,999,county,Durham County,ZSFH,1997-02-28,139430.0
1,999,county,Durham County,ZSFH,1997-03-31,139459.0
2,999,county,Durham County,ZSFH,1997-04-30,139659.0
3,999,county,Durham County,ZSFH,1997-05-31,139887.0
4,999,county,Durham County,ZSFH,1997-06-30,140303.0


In [287]:
# Deleting the word 'county' from zillow df

data_county_df['county'] = data_county_df['county'].str.replace(' County', '')

In [288]:
# Re-merge

master_df = pd.merge(data_county_df, county_coordinates_df, on='county')

In [289]:
master_df

,region_id,region_type,county,indicator_id,date,value,Sort [1],State,FIPS,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,999,county,Durham,ZSFH,1997-02-28,139430.0,1922.0,NC,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
1,999,county,Durham,ZSFH,1997-03-31,139459.0,1922.0,NC,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
2,999,county,Durham,ZSFH,1997-04-30,139659.0,1922.0,NC,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
3,999,county,Durham,ZSFH,1997-05-31,139887.0,1922.0,NC,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
4,999,county,Durham,ZSFH,1997-06-30,140303.0,1922.0,NC,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3166704,1000,county,Echols,ZSFH,2022-02-28,126118.0,437.0,GA,13101.0,Statenville,"4,034","1,074.57",414.894,15.087,5.825,"1,089.66",420.719,+30.708500°,–82.836132°
3166705,1000,county,Echols,ZSFH,2022-03-31,125100.0,437.0,GA,13101.0,Statenville,"4,034","1,074.57",414.894,15.087,5.825,"1,089.66",420.719,+30.708500°,–82.836132°
3166706,1000,county,Echols,ZSFH,2022-04-30,132115.0,437.0,GA,13101.0,Statenville,"4,034","1,074.57",414.894,15.087,5.825,"1,089.66",420.719,+30.708500°,–82.836132°
3166707,1000,county,Echols,ZSFH,2022-05-31,132054.0,437.0,GA,13101.0,Statenville,"4,034","1,074.57",414.894,15.087,5.825,"1,089.66",420.719,+30.708500°,–82.836132°
